In [38]:
import mysql.connector
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from pandas import Timestamp
from IPython.display import display
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.decision_tree import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
import spacy

creds = ["uktuserdaroglu","Vacy4201(Pri","172.20.20.4","hgo",3306]
#Connection to the database
host = creds[2]
user = creds[0]
password = creds[1]
database = creds[3]
port = creds[4]
mydb = mysql.connector.connect(host=host, user=user, database=database, port=port, password=password, auth_plugin='mysql_native_password')
mycursor = mydb.cursor()

#Safecheck to guarantee that the connection worked
mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
print(mydb.connection_id) #it'll give connection_id,if got connected

Tables: [('ConsultaUrgencia_doentespedidosconsultaNeurologia2012',), ('consultaneurologia2012',), ('consultaneurologia201216anon_true',), ('hgo_data_032023',)]
600


In [39]:
alertP = pd.read_sql("""SELECT * FROM consultaneurologia201216anon_true""",mydb)
new_data=pd.read_sql("""SELECT * FROM hgo_data_032023""",mydb)

/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
/Users/utkuserdaroglu/opt/anaconda3/lib/python3.9/site-packages/pandas/io/sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [40]:
alertP.info()
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10150 entries, 0 to 10149
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_DOENTE            10150 non-null  object
 1   PROCESSO             10150 non-null  object
 2   COD_REFERENCIA       10150 non-null  object
 3   COD_PZ               10150 non-null  object
 4   COD_UNID_SAUDE_PROV  10150 non-null  int64 
 5   UNID_PROV            10150 non-null  object
 6   TIPO_UNID            10150 non-null  object
 7   PROVENIENCIA         10150 non-null  object
 8   COD_CTH_PRIOR        10150 non-null  object
 9   CTH_PRIOR            10150 non-null  object
 10  COD_MOTIVO_RECUSA    10150 non-null  int64 
 11  DES_MOTIVO_RECUSA    10150 non-null  object
 12  COD_ESPECIALIDADE    10150 non-null  int64 
 13  DES_ESPECIALIDADE    10150 non-null  object
 14  agrupadora           10150 non-null  object
 15  OUTRA_ENTIDADE       10150 non-null  int64 
 16  DATA

In [41]:
new_data.rename(columns={"PZ = COD_Pz":'COD_PZ'},inplace=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Column1              362 non-null    int64 
 1   COD_PZ               362 non-null    object
 2   num_seq = ID_Doente  362 non-null    object
 3   text                 362 non-null    object
dtypes: int64(1), object(3)
memory usage: 11.4+ KB


In [42]:
#replacing "" texto strings with Nan value
alertP['Texto'] = alertP['Texto'].replace("", np.nan)


In [43]:
alertP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10150 entries, 0 to 10149
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_DOENTE            10150 non-null  object
 1   PROCESSO             10150 non-null  object
 2   COD_REFERENCIA       10150 non-null  object
 3   COD_PZ               10150 non-null  object
 4   COD_UNID_SAUDE_PROV  10150 non-null  int64 
 5   UNID_PROV            10150 non-null  object
 6   TIPO_UNID            10150 non-null  object
 7   PROVENIENCIA         10150 non-null  object
 8   COD_CTH_PRIOR        10150 non-null  object
 9   CTH_PRIOR            10150 non-null  object
 10  COD_MOTIVO_RECUSA    10150 non-null  int64 
 11  DES_MOTIVO_RECUSA    10150 non-null  object
 12  COD_ESPECIALIDADE    10150 non-null  int64 
 13  DES_ESPECIALIDADE    10150 non-null  object
 14  agrupadora           10150 non-null  object
 15  OUTRA_ENTIDADE       10150 non-null  int64 
 16  DATA

In [44]:
#checking the missing texto numbers in the dataset
missing_count = alertP['Texto'].isna().sum()
missing_count



8377

In [45]:
#Create a dictionary mapping reference codes to fill values
fill_dict = dict(zip(new_data['COD_PZ'], new_data['text']))



In [46]:
alertP.loc[alertP['Texto'].isna() & alertP['COD_PZ'].isin(fill_dict.keys()), 'Texto'] = alertP['COD_PZ'].map(fill_dict)


In [47]:
alertP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10150 entries, 0 to 10149
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID_DOENTE            10150 non-null  object
 1   PROCESSO             10150 non-null  object
 2   COD_REFERENCIA       10150 non-null  object
 3   COD_PZ               10150 non-null  object
 4   COD_UNID_SAUDE_PROV  10150 non-null  int64 
 5   UNID_PROV            10150 non-null  object
 6   TIPO_UNID            10150 non-null  object
 7   PROVENIENCIA         10150 non-null  object
 8   COD_CTH_PRIOR        10150 non-null  object
 9   CTH_PRIOR            10150 non-null  object
 10  COD_MOTIVO_RECUSA    10150 non-null  int64 
 11  DES_MOTIVO_RECUSA    10150 non-null  object
 12  COD_ESPECIALIDADE    10150 non-null  int64 
 13  DES_ESPECIALIDADE    10150 non-null  object
 14  agrupadora           10150 non-null  object
 15  OUTRA_ENTIDADE       10150 non-null  int64 
 16  DATA

In [48]:
1929-1773

156

In [49]:
merged_df = pd.merge(alertP, new_data, on=["COD_PZ"], how='right')

In [50]:
indexes = (merged_df['Texto'] != merged_df['text'])
merged_df[indexes]

,ID_DOENTE,PROCESSO,COD_REFERENCIA,COD_PZ,COD_UNID_SAUDE_PROV,UNID_PROV,TIPO_UNID,PROVENIENCIA,COD_CTH_PRIOR,CTH_PRIOR,...,Mês_entrada,Ano_entrada,trata data recusa,resume saída,mês_saida,ano_saida,Texto,Column1,num_seq = ID_Doente,text
5,NaN,NaN,NaN,GSGFEFD+B23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,ZZZZZD,"Cefaleias Paula Costa,Drª (Medicina Geral e Fa..."
11,SSHCCB,LVMVDV,LQHSOQLH,GDCADAB,3151575.0,USF TORRE DA MARINHA,CS/USF,CTH,3,Normal,...,11.0,2014.0,21/05/2015,12/06/2015,6,2015,"Utente de 75 anos, refere tremor desde há 5 an...",11,SSHCCB,"Utente de 75 anos, refere tremor desde há 5 an..."
15,ZBCFFD,LIEMBE,LQHRQLUS,GHCDBFH,3150672.0,USF FEIJO-CS COVA DA PIEDADE,CS/USF,CTH,3,Normal,...,10.0,2014.0,31/10/2014,24/11/2014,11,2014,SEGUNDO A ESPOSA ESQUECE DO QUE FAZ AGORA ABRE...,15,ZBCFFD,"DEMÊNCIA Carolina Pereira,Drª (Medicina Geral ..."
18,FEZCGS,LLCNMCN,LQHQQVQL,BBGDEAS,3151571.0,USF FFMAIS-CS SEIXAL,CS/USF,CTH,3,Normal,...,8.0,2014.0,16/04/2015,20/04/2015,4,2015,"Sintomatologia: doente 76 anos, já referenciad...",18,FEZCGS,"s. demencial? Paula Costa, Drª. (Medicina Gera..."
19,NaN,NaN,NaN,BBCGGHHHHH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,ZFCBDF,Desde há 2 anos quadro clinico progressivo de ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,CFC00000000,ILBMVIJ,LSHQSPRU,ZDFESGCC,3150605.0,UCSP STO. ANTONIO DO LARAJEIRO,CS/USF,CTH,3,Normal,...,7.0,2016.0,03/08/2016,19/10/2016,10,2016,"Justificação PARKINSONISMO ? Isabel Rodrigues,...",352,CFC00000000,"PARKINSONISMO ? Isabel Oliveira, Drª. (Medicin..."
357,ESSAA,LIJVJ,LSHPSHSP,ZDEHEZZE,3150671.0,USF COVA DA PIEDADE-CS C PIEDA,CS/USF,CTH,3,Normal,...,5.0,2016.0,06/07/2016,07/09/2016,9,2016,tremor das mãos com com cerca de um ano de evo...,357,ESSAA,"Parkinsonismo?? Joana Costa , Drª. (Medicina G..."
358,AZHBBD,MILBDLL,LSHPRLHT,ZDDGZAGD,3151574.0,USF CSI SEIXAL,CS/USF,CTH,3,Normal,...,5.0,2016.0,06/07/2016,09/08/2016,8,2016,Homem de 76 anos com perturbações da marcha e ...,358,AZHBBD,Homem de 76 anos com perturbações da marcha e ...
359,FDCCBE,LLCLLBB,LSHPOQHP,ZDDEAEZF,3150371.0,USF S JOAO DO PRAGAL-CS ALMADA,CS/USF,CTH,3,Normal,...,5.0,2016.0,16/06/2016,29/08/2016,8,2016,80 anos com hemiparesia espástica à esq. após ...,359,FDCCBE,80 anos com hemiparesia espástica à esq. após ...


In [51]:
#new_data['res'] = np.where(new_data["COD_PZ"]==alertP["COD_PZ"],1,0)

In [52]:
merged_df = pd.merge(alertP, new_data, on=["COD_PZ"], how='left')
unmatched_df = merged_df[merged_df['Texto'].isna()]

